<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%BA%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-generativeai

In [ ]:
import gradio as gr
import pandas as pd
from google.colab import auth
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import os
import json

In [ ]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('repo')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
# === 連線到試算表 ===
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1UIfts0iHJzLn6VdOeuT3WS7UKDEdS5dylr9WxK1BhFA/edit?gid=0#gid=0"
WORKSHEET_NAME = "工作表2"

REQUIRED_COLUMNS = ["日期", "科目", "作業成績"]

_auth_done = False
_gc = None
_ws = None


In [ ]:
# --- 主要功能區塊 ---
def get_user_grades():
    """
    透過終端機輸入學生成績，直到使用者輸入 'q' 結束。
    """
    print("--- 準備輸入成績。輸入 'q' 來停止。---")
    grades = []
    while True:
        subject = input("請輸入科目（或輸入 'q' 停止）：")
        if subject.lower() == 'q':
            break

        grade = input(f"請輸入 {subject} 的成績：")
        try:
            grade = int(grade)
        except ValueError:
            print("成績必須是數字。請重新輸入。")
            continue

        today = datetime.now().strftime('%Y-%m-%d')
        grades.append([today, subject, grade])
        print(f"已記錄：日期: {today}, 科目: {subject}, 成績: {grade}\n")

    return grades

In [ ]:
def get_ai_summary(grades):
    """
    呼叫 Gemini 模型來生成成績摘要與常見迷思。
    """
    model = genai.GenerativeModel('gemini-1.5-flash')

    # 準備給 AI 的提示
    prompt_text = "以下是學生的成績列表，請幫我根據這些成績，產出一個簡單的摘要與常見迷思整理（不評分，只做總結）。\n\n"
    for record in grades:
        date, subject, grade = record
        prompt_text += f"日期：{date}, 科目：{subject}, 成績：{grade}\n"

    print("\n--- 正在呼叫 AI 模型生成摘要... ---")
    try:
        response = model.generate_content(prompt_text)
        summary = response.text
        return summary
    except Exception as e:
        print(f"呼叫 AI 時發生錯誤：{e}")
        return "AI 摘要生成失敗。"

In [ ]:
def main():
    """
    主程式流程：輸入成績 -> 獲取 AI 摘要 -> 寫入 Google Sheet。
    """
    try:
        # 1. Google Sheet 身份驗證
        auth.authenticate_user()

        creds, _ = default()
        gc = gspread.authorize(creds)

        sh = gc.open_by_url(SPREADSHEET_URL)
        ws = sh.worksheet(WORKSHEET_NAME)





        print("--- Google Sheet 連線成功。---")

        # 2. 獲取使用者輸入的成績
        new_grades = get_user_grades()

        if not new_grades:
            print("沒有輸入任何成績，程式結束。")
            return

        # 3. 將新成績寫入 Google Sheet
        ws.append_rows(new_grades)
        print("\n--- 成績已成功寫入 Google Sheet。---")

        # 4. 獲取 AI 摘要並寫入 Google Sheet
        summary = get_ai_summary(new_grades)

        # 尋找第一行空白列
        next_row = len(ws.col_values(1)) + 1

        # 使用 update_cell() 方法逐一更新儲存格
        ws.update_cell(next_row, 1, datetime.now().strftime('%Y-%m-%d'))
        ws.update_cell(next_row, 2, 'AI 摘要')

        # 為了避免單元格內容過長，將摘要內容分成多行來寫入
        summary_lines = summary.split('\n')
        for i, line in enumerate(summary_lines):
            ws.update_cell(next_row + i, 3, line)

        print("\n--- AI 摘要已成功寫入 Google Sheet。---")
        print("以下是 AI 生成的摘要內容：")
        print("-" * 50)
        print(summary)
        print("-" * 50)

    except gspread.exceptions.APIError as e:
        print(f"Google Sheets API 錯誤：{e.response.text}")
        print("請確認：")
        print("1. 您的服務帳戶金鑰檔案正確且未過期。")
        print("2. 您已將服務帳戶的 Email 地址（在 JSON 檔案中）分享給 Google Sheet，並給予編輯權限。")
    except Exception as e:
        print(f"發生未預期的錯誤：{e}")

if __name__ == "__main__":
    main()

--- Google Sheet 連線成功。---
--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：數學
請輸入 數學 的成績：80
已記錄：日期: 2025-09-25, 科目: 數學, 成績: 80

請輸入科目（或輸入 'q' 停止）：q

--- 成績已成功寫入 Google Sheet。---

--- 正在呼叫 AI 模型生成摘要... ---

--- AI 摘要已成功寫入 Google Sheet。---
以下是 AI 生成的摘要內容：
--------------------------------------------------
**摘要：**

2025年9月25日數學考試成績為80分。


**常見迷思整理 (針對單一數據無法提出)：**

由於僅提供一筆單一科目的成績數據，無法推論出任何常見的學習迷思。  要整理常見迷思，需要更多數據，例如：不同科目的成績、歷次考試的成績趨勢、學生自我評估、學習習慣等資訊。

--------------------------------------------------
